# Neo4j Database Connection Tutorial for kindmesh

This notebook demonstrates how to connect to the Neo4j database used by kindmesh and run basic queries.

## Setup

First, let's install the required packages if they're not already installed:

In [ ]:
!pip install neo4j pandas

## Importing the Required Libraries

In [ ]:
import os
import pandas as pd
from neo4j import GraphDatabase
from datetime import datetime

## Connecting to Neo4j

We'll create a simple class to handle Neo4j connections:

In [ ]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.uri = uri
        self.user = user
        self.password = password
        self.driver = None
        self.connect()
        
    def connect(self):
        self.driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))
        # Test connection
        with self.driver.session() as session:
            result = session.run("RETURN 'Connection successful' as message")
            print(result.single()["message"])
    
    def close(self):
        if self.driver:
            self.driver.close()
    
    def query(self, query, parameters=None):
        if not parameters:
            parameters = {}
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record for record in result]
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

## Establishing a Connection

Now let's connect to the Neo4j database:

In [ ]:
# Connection parameters - adjust these to match your Neo4j setup
uri = "bolt://localhost:7687"  # Change if your Neo4j is hosted elsewhere
user = "neo4j"
password = "kindmesh"  # Use your actual password

# Connect to Neo4j
conn = Neo4jConnection(uri, user, password)

## Running Basic Queries

Let's run some basic queries to explore the database:

In [ ]:
# Get database schema
query = """
CALL db.schema.visualization()
"""
result = conn.query(query)
print("Database schema retrieved")

In [ ]:
# Count nodes by label
query = """
MATCH (n)
RETURN labels(n) AS Node_Type, count(*) AS Count
ORDER BY Count DESC
"""
result = conn.query(query)

# Convert to DataFrame for better display
df = pd.DataFrame([{"Node_Type": r["Node_Type"][0] if r["Node_Type"] else "None", 
                    "Count": r["Count"]} for r in result])
df

## Retrieving User Data

Let's retrieve information about users in the system:

In [ ]:
# Get all users (excluding password hashes for security)
query = """
MATCH (u:User)
RETURN u.username AS Username, u.role AS Role, 
       u.created_at AS Created_At, u.created_by AS Created_By
ORDER BY u.created_at DESC
"""
result = conn.query(query)

# Convert to DataFrame
users_df = pd.DataFrame([dict(r) for r in result])
users_df

## Retrieving Interaction Data

Let's retrieve recent interactions:

In [ ]:
# Get recent interactions
query = """
MATCH (u:User)-[i:LOGGED]->(interaction:Interaction)-[:WITH]->(r:Recipient)
RETURN u.username AS Logged_By, 
       interaction.type AS Interaction_Type,
       interaction.timestamp AS Timestamp,
       r.key AS Recipient_Key,
       r.pseudonym AS Recipient_Pseudonym,
       interaction.notes AS Notes
ORDER BY interaction.timestamp DESC
LIMIT 10
"""
result = conn.query(query)

# Convert to DataFrame
interactions_df = pd.DataFrame([dict(r) for r in result])
interactions_df

## Creating an Index

Let's create an index to improve query performance:

In [ ]:
# Create an index on Recipient.key for faster lookups
query = """
CREATE INDEX recipient_key IF NOT EXISTS FOR (r:Recipient) ON (r.key)
"""
conn.query(query)
print("Index created successfully")

## Closing the Connection

Always close the connection when you're done:

In [ ]:
# Close the connection
conn.close()
print("Connection closed")